In [8]:
import os
import sys
from pathlib import Path

# Ensure project root is on sys.path so 'backtester' imports work
cwd = Path(os.getcwd())
candidates = [cwd, cwd.parent, Path("..").resolve()]
for p in candidates:
    if (p / "backtester").exists() and str(p) not in sys.path:
        sys.path.insert(0, str(p))
        break

print("PYTHONPATH set. Using root:", sys.path[0])

PYTHONPATH set. Using root: C:\Users\User\Desktop\Crypto strategy backtest


## Load Data

In [9]:
import pandas as pd

from backtester.models import Position, Side, ActionType
from backtester.strategy_base import StrategyContext

from backtester.strategies.ALBO_strategy import ALBOStrategy, ALBOParams


In [5]:
idx = pd.date_range("2026-01-01", periods=10, freq="5min")
idx

DatetimeIndex(['2026-01-01 00:00:00', '2026-01-01 00:05:00',
               '2026-01-01 00:10:00', '2026-01-01 00:15:00',
               '2026-01-01 00:20:00', '2026-01-01 00:25:00',
               '2026-01-01 00:30:00', '2026-01-01 00:35:00',
               '2026-01-01 00:40:00', '2026-01-01 00:45:00'],
              dtype='datetime64[ns]', freq='5min')

In [10]:

def load_crypto_parquet_data(coin_name: str, timeframe: str = "5m", nM: int = 54) -> pd.DataFrame:
    df = pd.read_parquet(f'{coin_name}_{timeframe}_{nM}M_UTC.parquet')
    return df
def generate_us_session_bars_info(df):
    # 確保時間有時區資訊
    df['dt_ny'] = pd.to_datetime(df['dt_utc'], utc=True).dt.tz_convert('America/New_York')

    # 取日期（當地日曆）
    df['date'] = df['dt_ny'].dt.date
    df['weekday'] = df['dt_ny'].dt.day_name() 
    # 對每天依時間排序並編號
    df = df.sort_values(['date', 'dt_ny']).reset_index(drop=True)
    df['bar_index'] = df.groupby('date').cumcount() + 1  # 第幾根K線，從1開始
    df = df.loc[df['weekday'].isin(['Monday','Tuesday','Wednesday','Thursday','Friday'])]
    # 查看結果
    print(df[['dt_ny', 'date', 'weekday', 'bar_index']].head(5))
    return df
def generate_allday_bars_info(df):
# 確保時間有時區資訊
    df['dt_ny'] = pd.to_datetime(df['dt_utc'], utc=True).dt.tz_convert('America/New_York')

    # 取日期（當地日曆）
    df['date'] = df['dt_ny'].dt.date
    df['time'] = df['dt_ny'].dt.time
    df['weekday'] = df['dt_ny'].dt.day_name() 
    # 對每天依時間排序並編號
    df = df.sort_values(['date', 'dt_ny']).reset_index(drop=True)
    df['bar_index'] = df.groupby('date').cumcount() + 1  # 第幾根K線，從1開始
    df = df.loc[df['weekday'].isin(['Monday','Tuesday','Wednesday','Thursday','Friday'])]#, 'Saturday','Sunday'
    # 查看結果
    print(df[['dt_ny', 'date', 'weekday', 'bar_index']].head(5))
    return df

In [ ]:
coin_name = 'BTC'  # 可更改為 'ETH', 'SOL', 'BTC', 'ADA', 'PAXG' 等等
nM = 54

df = pd.read_parquet(f'{coin_name}_5m_{nM}M_UTC.parquet')
df_1m = pd.read_parquet(f'{coin_name}_1m_{nM}M_UTC.parquet')


## 